This is a test Jupyter Notebook for Dr. Byrum's RNASeq Analysis Pipeline

To install R in Jupyter Notebook, use the following command from the Anaconda terminal.
conda install -c r r-irkernel

In [1]:
# Install necessary packages into R environment
install.packages("matrixStats")

install.packages("BiocManager")
library("BiocManager")

BiocManager::install("edgeR", quietly=TRUE)
BiocManager::install("AnnotationDbi", quietly=TRUE)
BiocManager::install("org.Mm.eg.db", quietly=TRUE)
BiocManager::install("Glimma")

Installing package into 'C:/Users/3062742/Documents/R/win-library/3.6'
(as 'lib' is unspecified)


package 'matrixStats' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\3062742\AppData\Local\Temp\Rtmpsj5i3p\downloaded_packages


Installing package into 'C:/Users/3062742/Documents/R/win-library/3.6'
(as 'lib' is unspecified)


package 'BiocManager' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\3062742\AppData\Local\Temp\Rtmpsj5i3p\downloaded_packages


Warning message:
"package 'BiocManager' was built under R version 3.6.3"Bioconductor version 3.10 (BiocManager 1.30.10), ?BiocManager::install for help
Bioconductor version '3.10' is out-of-date; the current release version '3.11'
  is available with R version '4.0'; see https://bioconductor.org/install
Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.1 (2019-07-05)
Installing package(s) 'edgeR'


...

Old packages: 'rlang', 'digest', 'evaluate', 'htmltools', 'IRkernel',
  'jsonlite', 'Rcpp', 'repr', 'uuid'
Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.1 (2019-07-05)
Installing package(s) 'AnnotationDbi'


package 'AnnotationDbi' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\3062742\AppData\Local\Temp\Rtmpsj5i3p\downloaded_packages


Old packages: 'rlang', 'digest', 'evaluate', 'htmltools', 'IRkernel',
  'jsonlite', 'Rcpp', 'repr', 'uuid'
Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.1 (2019-07-05)
Installing package(s) 'org.Mm.eg.db'
installing the source package 'org.Mm.eg.db'

Warning message in install.packages(...):
"installation of package 'org.Mm.eg.db' had non-zero exit status"Old packages: 'rlang', 'digest', 'evaluate', 'htmltools', 'IRkernel',
  'jsonlite', 'Rcpp', 'repr', 'uuid'
Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.1 (2019-07-05)
Installing package(s) 'Glimma'


package 'Glimma' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\3062742\AppData\Local\Temp\Rtmpsj5i3p\downloaded_packages


Old packages: 'rlang', 'digest', 'evaluate', 'htmltools', 'IRkernel',
  'jsonlite', 'Rcpp', 'repr', 'uuid'


In [2]:
library(edgeR, quietly-TRUE)

# import targets and counts
targets <- read.csv("targets.csv", header=TRUE, stringsAsFactors=FALSE); head(targets)

GenewiseCounts <- read.csv("htseq_counts_12.csv", header=TRUE, row.names=1, stringsAsFactors=FALSE)
head(GenewiseCounts)

Warning message:
"package 'edgeR' was built under R version 3.6.2"Loading required package: limma
Warning message:
"package 'limma' was built under R version 3.6.2"

sample,group,batch
D1,CON,b1
D2,CON,b1
D3,CON,b1
D4,CON,b1
D5,CON,b1
D6,CON,b2


,D1,D2,D3,D4,D5,D6,E1,E2,E3,E4,E5,E6
Adora2b,0,2,0,2,0,2,8,3,1,5,7,40
Adora2a,731,1036,749,913,761,334,825,1298,1321,1137,1096,617
Gm20741,0,0,0,0,0,0,0,0,0,0,0,0
Gm4340,0,0,0,0,0,0,0,0,0,0,0,0
Gm20744,0,0,0,0,0,0,0,0,0,0,0,0
Gm20743,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
# define main factors for the analysis (group main factor for analysis, batch, etc)
targets$group <- factor(targets$group, levels=c(unique(targets$group))); table(targets$group); levels(targets$group)
targets$batch <- factor(as.character(targets$batch), levels=c(unique(targets$batch))); table(targets$batch); levels(targets$batch)

# create data.frame with factors (potential factors for the analysis)
my_targets <- data.frame(group=targets$group, batch=targets$batch);
rownames(my_targets) <- targets$sample
head(my_targets)


table(my_targets$group)
table(my_targets$batch)


CON EPZ 
  6   6 

[1] "CON" "EPZ"


b1 b2 
10  2 

[1] "b1" "b2"

,group,batch
D1,CON,b1
D2,CON,b1
D3,CON,b1
D4,CON,b1
D5,CON,b1
D6,CON,b2



CON EPZ 
  6   6 


b1 b2 
10  2 

In [4]:
# create DGEList()
y <- DGEList(counts=GenewiseCounts, group=  targets$group, genes=rownames(GenewiseCounts), remove.zeros=FALSE)
colnames(y$genes)[1] <- "gene_id"
head(y$counts)[,1:5];head(y$genes);head(y$samples)


print("used DGEList()")

,D1,D2,D3,D4,D5
Adora2b,0,2,0,2,0
Adora2a,731,1036,749,913,761
Gm20741,0,0,0,0,0
Gm4340,0,0,0,0,0
Gm20744,0,0,0,0,0
Gm20743,0,0,0,0,0


,gene_id
Adora2b,Adora2b
Adora2a,Adora2a
Gm20741,Gm20741
Gm4340,Gm4340
Gm20744,Gm20744
Gm20743,Gm20743


,group,lib.size,norm.factors
D1,CON,22385619,1
D2,CON,28092434,1
D3,CON,19372856,1
D4,CON,27283589,1
D5,CON,23735343,1
D6,CON,17215483,1


[1] "used DGEList()"


In [ ]:
library(AnnotationDbi, quietly=TRUE)
library(org.Mm.eg.db, quietly=TRUE)


org.Mm.eg.db
keytypes(org.Mm.eg.db)


y$genes$ENSEMBL  <- mapIds(org.Mm.eg.db, keys=rownames(y$genes), keytype = "SYMBOL", column="ENSEMBL", multiVals="first")
y$genes$ENTREZID <- mapIds(org.Mm.eg.db, keys=rownames(y$genes), keytype = "SYMBOL", column="ENTREZID", multiVals="first")
y$genes$SYMBOL   <- mapIds(org.Mm.eg.db, keys=rownames(y$genes), keytype = "SYMBOL", column="SYMBOL", multiVals="first")
y$genes$GENENAME <- mapIds(org.Mm.eg.db, keys=rownames(y$genes), keytype = "SYMBOL", column="GENENAME", multiVals="first")


# final imported data
head(y$counts)[,1:5]
head(y$samples)
head(y$genes)


print("added org.Mm.eg.db")

In [ ]:
## [2] REMOVE GENES WITH ZERO COUNTS

#---------------------------------------------------------------------------------------------------------
print(paste("[",Sys.time(),"]"," removing genes with 0 counts in all samples... ", sep=""))
#---------------------------------------------------------------------------------------------------------
# number genes with zero counts in all samples
table(rowSums(y$counts==0)==ncol(y$counts))

# genes with zero counts in all samples
zeros <- rowSums(y$counts)==0; table(zeros)


# keep genes with at least 1 count in 1 sample (i.e. filter genes with zero counts in all samples). 
keep <- rowSums(y$counts) > 0; table(keep)
y <- y[keep,, keep.lib.sizes=FALSE];dim(y)


##--------------------
##   RAW VARIABLES
##--------------------
y.raw          <- y
ngroups        <- length(unique(y$samples$group)); ngroups
nsamples       <- ncol(y); nsamples
ngenes         <- nrow(y); ngenes

raw_lib        <- y$samples; head(raw_lib)
raw_counts     <- y$counts; dim(raw_counts)
raw_cpm        <- cpm(y, prior.count=2); dim(raw_cpm)
raw_lcpm       <- cpm(y, prior.count=2, log=TRUE); dim(raw_lcpm)
raw_anno       <- y$genes; head(raw_anno)
raw_L          <- log2(2/(mean(y$samples$lib.size) * 1e-06)); raw_L

pearson.raw_lcpm      <- cor(raw_lcpm, use="all.obs", method="pearson")
spearman.raw_lcpm     <- cor(raw_lcpm, use="all.obs", method="spearman")

raw_var     <- matrixStats::rowVars(raw_lcpm) # row variance per gene
raw_scale   <- t(scale(t(raw_lcpm)))  # scaled lcpm values mean=0, std=1

print("Saved raw variables")

stopifnot(rownames(raw_scale)==rownames(y$genes))
stopifnot(rownames(raw_scale)==rownames(y$counts))

In [ ]:
## mean, median, min., and max. library sizes
L  <- mean(y$samples$lib.size) * 1e-06
M  <- median(y$samples$lib.size) * 1e-06
Mn <- min(y$samples$lib.size) * 1e-06
Mx <- max(y$samples$lib.size) * 1e-06
round(c(L, M, Mn, Mx),1)



# smallest libraries
min_group <- min(table(targets$group))  # 6
min_group
 
# calculates the CPM value that corresponds to a count of 10
cpm(10, min(y$samples$lib.size))
min_cpm <- round(as.numeric(cpm(10, min(y$samples$lib.size))),2)
min_cpm # 0.58

keep <- rowSums(cpm(y) > min_cpm) >=  min_group
table(keep)
# FALSE  TRUE 
#  5136 11457


y <- y[keep, , keep.lib.sizes=FALSE]
dim(y) 

# The option keep.lib.sizes=FALSE causes the library sizes to be recomputed after the filtering.
# This is generally recommended, although the effect on the downstream analysis is usually small.



##---------------------------
## FILTER VARIABLES (y)
##---------------------------
y.filter           <- y
filter_lib         <- y$samples
filter_ngenes      <- nrow(y); filter_ngenes
filter_anno        <- y$genes;head(filter_anno)
head(filter_anno)[1:5,];dim(filter_anno)
filter_counts      <- y$counts
filter_cpm         <- cpm(y, prior.count=2)
filter_lcpm        <- cpm(y, prior.count=2, log=TRUE)
filter_L           <- log2(2/(mean(y$samples$lib.size) * 1e-6));filter_L 

pearson.filter_lcpm      <- cor(filter_lcpm, use="all.obs", method="pearson")
spearman.filter_lcpm     <- cor(filter_lcpm, use="all.obs", method="spearman")

filter_var     <- matrixStats::rowVars(filter_lcpm) # row variance per gene
filter_scale   <- t(scale(t(filter_lcpm)))  # scaled lcpm values mean=0, std=1

print("Saved filtered variables")

stopifnot(rownames(filter_scale)==rownames(y$genes))
stopifnot(rownames(filter_scale)==rownames(y$counts))

In [ ]:
##-------------------------------
##     TMM NORMALIZATION (y)     
##-------------------------------

# NORMALIZED DATA (y)
y <- calcNormFactors(y, method="TMM")
head(y$samples)




##---------------------------
##   NORM VARIABLES (y)
##---------------------------
y.norm          <- y
norm_lib        <- y$samples
norm_lib        <- cbind(y$samples, (y$samples$norm.factors * y$samples$lib.size))
colnames(norm_lib) <- c("group", "old.lib.size", "norm.factors", "lib.size"); head(norm_lib)

norm_factor     <- y$samples$norm.factors
norm_anno       <- y$genes
norm_counts     <- y$counts
norm_cpm        <- cpm(y, prior.count=2, normalized.lib.sizes = TRUE)
norm_lcpm       <- cpm(y, prior.count=2, normalized.lib.sizes = TRUE, log=TRUE)
norm_L          <- log2(2/(mean(y$samples$lib.size, normalized.lib.sizes=TRUE) * 1e-06)); norm_L 

pearson.norm_lcpm      <- cor(norm_lcpm, use="all.obs", method="pearson")
spearman.norm_lcpm     <- cor(norm_lcpm, use="all.obs", method="spearman")

norm_var     <- matrixStats::rowVars(norm_lcpm) # row variance per gene
norm_scale   <- t(scale(t(norm_lcpm)))  # scaled lcpm values mean=0, std=1

print("Saved normalized variables")

stopifnot(rownames(norm_scale)==rownames(y$genes))
stopifnot(rownames(norm_scale)==rownames(y$counts))

In [ ]:
design = model.matrix(~0 + my_targets$group + my_targets$batch,  data= y$samples)
colnames(design) <- c("CON","EPZ", "b2");design

contrasts <- makeContrasts(EPZ_vs_CON=EPZ-CON, levels=colnames(design));contrasts


v <- voom(y, design, plot=FALSE)
vfit<-lmFit(v, design)
vfit<-contrasts.fit(vfit,contrasts=contrasts)
results <-eBayes(vfit)


png(file.path(".", "01_voom.png"),units="in", width=10, height=5, res=1000, pointsize=12)
par(mfrow=c(1,2))
   voom(y,design=design, normalize.method="none", plot=TRUE, save.plot=TRUE)
   plotSA(results, main="Final Model")
dev.off()


stats <- topTable(results, coef=1, n=Inf, adjust.method="BH", sort.by="none", p.value=1, lfc=0); dim(stats)
results$stats <- stats
sig <- topTable(results, coef=1, n=Inf, adjust.method="BH", sort.by="none", p.value=0.05, lfc=1); dim(sig)
dt <- decideTests(results, method="separate", adjust.method="BH", p.value=0.05, lfc = 1)
summary(dt)
#        EPZ_vs_CON
# Down            7
# NotSig      10738
# Up            712

In [ ]:
png(file.path(".", paste("05_","EPZ_vs_CON", "_p-value.histogram.png",sep="")), units="in", width=6, height=5, res=1000, pointsize=12)
{
   grayblue <-"#b0ddf5"
      
     hist(results$stats$P.Value, col= grayblue, breaks = 100, 
          main=paste("P-value Histogram: ","EPZ vs. CON",sep=""),
          xlab="P.Value", 
          ylab="frequency", 
          font=1, cex.main=1, las=1)
}
dev.off()

In [ ]:
i<-1

png(file.path(".", paste("07_","EPZ_vs_CON","_MD.plot.png",sep="")), units="in", width=5, height=5, res=1000, pointsize=10)
{
     ##----------------------------------------------------------------------------
     op <- par(no.readonly = TRUE)  # save current par settings b4 plotting
     #par(mfrow = c(4, 5))          # multiple plots
     par(mar = c(5, 5, 4, 1))       # inner margin (bottom, left, top, right)
     par(mgp = c(3, 1, 0))          # axis margin (labels, ticks, line)
     par(oma = c(2, 1, 0, 1))       # outer margin (bottom, left, top, right)
     ##----------------------------------------------------------------------------
     plotMD(results, column=i, cex=1, las=1, status=dt[,i], values=c(1,-1), col=c("red","blue"), 
            legend="topright", main="", 
            xlab=expression(paste("avg. ", log[2]," (counts-per-million)",sep="")),
            ylab=expression(paste(log[2]," (fold-change)",sep="")),
            cex.axis=1.1, cex.lab=1.2
            # ,ylim=c(-11,13)
            #,xlim=c(2,14)
     )
     title(main=paste("MD plot: ", "EPZ vs. CON", "", sep=""))
     legend("topright", c("up", "not sig.", "down"), inset=0, pch=19,
            box.col="black", box.lwd=1, bg="white",
            col=c("red","black","blue"), ncol=1, cex=1)
     
     # text(x=12, y=-3, label=paste("up =", summary(dt)[3], sep=" "), adj=0, col="red", cex=0.9, font=1)
     # text(x=12, y=-4, label=paste("not sig. =", summary(dt)[2], sep=" "), adj=0, col="black", cex=0.9, font=1)
     # text(x=12, y=-5, label=paste("down =", summary(dt)[1], sep=" "), adj=0, col="blue", cex=0.9, font=1)
     abline(h=c(-1,1), col="black", lty=2,lwd=1)
}


dev.off()


In [ ]:
library(Glimma, quietly=TRUE)
  
glMDPlot(results, 
         coef=i,
         counts = norm_lcpm, 
         anno = results$genes,
         groups = my_targets$group, 
         samples = colnames(norm_lcpm), 
         status = dt[,i],
         transform=FALSE, 
         main = paste("MD Plot: ","EPZ vs. CON", "", sep=" "),
         xlab = "avg. log2 (counts-per-million)",
         ylab = "log2 (fold-change)", 
         side.xlab = "group",
         side.ylab = "expression (norm_lcpm)", 
         side.log = FALSE,
         # side.gridstep = ifelse(!transform || side.log, FALSE, 0.5),
         p.adj.method = "BH",
         jitter = 30, 
         side.main = "SYMBOL",
         display.columns = colnames(results$genes), 
         cols = c("#00bfff", "#858585", "#ff3030"),
         #sample.cols = colors[my_targets$group],
         path = file.path("."),
         folder = "MD-plots", 
         html = paste("EPZ_vs_CON", "_MD.plot", sep=""), 
         launch = FALSE
     )

In [ ]:
png(file.path(".", paste("08_","EPZ_vs_CON","_volcano.plot.png",sep="")), units="in", width=6, height=6, res=1000, pointsize=12)
{
     with(results$stats, 
          plot(logFC, -log10(adj.P.Val),# pch=20, las=1,
               pch = 21, bg = "black", col = "black", lwd = 0.9, cex = 1,
               main=paste("Volcano Plot: ", "EPZ vs. CON","", sep=" "), 
               xlab=expression(paste(,log[2]," (fold-change)",sep="")),
               #xlab= "logFC", 
               ylab=expression(paste("-",log[10]," (adj. p-value)",sep=""))
               # ylab="-log10 (P.Value)", 
               # ,ylim= c(0, 8)
               , xlim= c(-2,7)
               , cex.axis=1.1, cex.lab=1.2
          )
     )
     # grid()
     
     with(subset(results$stats, adj.P.Val <= 0.05 & logFC >= 1), points(logFC, -log10(adj.P.Val), pch = 21, bg = "firebrick2", col = "firebrick2", lwd = 1, cex = 1.1))
     with(subset(results$stats, adj.P.Val <= 0.05 & logFC <= -1), points(logFC, -log10(adj.P.Val), pch = 21, bg = "blue", col = "blue", lwd = 1, cex = 1.1))

     abline(v= c(-1,1), col="black", lty=2, lwd=1)
     abline(h= -log10(0.05), col="black", lty=2, lwd=1)
     
     # text(x=-14, y=8, label=paste("up = ", summary(dt)[3], sep=" "), adj=0, col="red", cex=0.8, font=1)
     # text(x=-14, y=7.5, label=paste("not sig = ", summary(dt)[2], sep=" "), adj=0, col="black", cex=0.8, font=1)
     # text(x=-14, y=7, label=paste("down = ", summary(dt)[1], sep=" "), adj=0, col="blue", cex=0.8, font=1)
}

dev.off()

In [ ]:
glXYPlot(x=results$stats$logFC,
         y=-log(results$stats$adj.P.Val, 10),
         counts = norm_lcpm,
         groups = my_targets$group,
         samples = rownames(my_targets),
         status = dt[,i], 
         transform=FALSE, 
         anno = results$genes,
         display.columns = colnames(results$genes),
         xlab = "log2 (fold-change)",
         ylab = "-log10 (adj. p-value)",
         side.main = "SYMBOL",
         side.xlab = "group",
         side.ylab = "expression (norm_lcpm)",
         side.log=FALSE,
         #sample.cols = colors[my_targets$group],
         cols = c("#00bfff", "#858585", "#ff3030"),
         p.adj.method="BH",
         jitter = 30,
         path = file.path("."),
         folder = "Volcano-plots",
         html = paste("EPZ_vs_CON","_volcano.plot", sep=""),
         main=paste("Volcano Plot: ","EPZ vs. CON", "", sep=" "),
         launch = FALSE
         )